In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations

In [2]:
df23after = pd.read_excel("2023-02-07_SP23_FinalExam_Data_Sendfile.xlsx", "Y & YO enrl", dtype={'Section': 'str', 'Catalog': 'str'})
df23bef = pd.read_excel("2023-01-09SP23FinalsDraftDataSend_1.11.23.xlsx", "Y & YO Enrl", dtype={'Section': 'str', 'Catalog': 'str'})
df22fa_bef= pd.read_excel("fa22_StdntEnrl_09-01-2022.xlsx", dtype={'Section': 'str', 'Catalog': 'str'})
df22fa_aft = pd.read_excel("fa22_StdntEnrl_09-10-2022.xlsx", dtype={'Section': 'str', 'Catalog': 'str'})
df22sp_bef= pd.read_excel("sp22_StdntEnrl_02-03-2022.xlsx", dtype={'Section': 'str', 'Catalog': 'str'})
df22sp_aft = pd.read_excel("sp22_StdntEnrl_02-10-2022.xlsx", dtype={'Section': 'str', 'Catalog': 'str'})

In [3]:
def processIfDrop(before, after):
    # process dataframe
    after['course'] = after['Subject'] +"-"+ after['Catalog']
    exam_after = after[(after['Final Exam'] == 'Y&YO') | (after['Final Exam'] == 'YO') | (after['Final Exam'] == 'Y')]
    exam_after = exam_after.sort_values(by=['PS Key']).reset_index()
    exam_after = exam_after[~exam_after['course'].isna()]
    exam_after['course'] = exam_after['course'].astype(str)
    exam_after = exam_after.rename(columns=lambda x: x + '-r')
    before['course'] = before['Subject'] +"-"+ before['Catalog']
    exam_bef = before[(before['Final Exam'] == 'Y&YO') | (before['Final Exam'] == 'YO') | (before['Final Exam'] == 'Y')]
    exam_bef = exam_bef.sort_values(by=['PS Key']).reset_index()
    exam_bef = exam_bef[~exam_bef['course'].isna()]
    exam_bef['course'] = exam_bef['course'].astype(str)
    
    # calculate If_Drop value and add the column to before dataframe
    merge = pd.merge(exam_bef, exam_after, left_on=['ID', 'Class Nbr'], right_on=['ID-r', 'Class Nbr-r'], how='left', indicator=True)
    # drop the class -> 1
    # keep the class -> -1
    merge['_merge'] = merge['_merge'].replace('left_only', 1)
    merge['_merge'] = merge['_merge'].replace('both', -1)
    merge=merge.rename(columns={'_merge': 'If_Drop'})
    columns = exam_bef.columns.tolist()
    columns.append('If_Drop')
    return merge[columns]



In [4]:
fa22 = processIfDrop(df22fa_bef, df22fa_aft)
sp22 = processIfDrop(df22sp_bef, df22sp_aft)
fa23 = processIfDrop(df23bef, df23after)

In [5]:
df_all = pd.concat([fa22, sp22, fa23], axis = 0)

In [6]:
df_all

,index,ID,Term,Class Nbr,Course ID,Offer Nbr,Session,Section,Subject,Catalog,...,PS Key,Final Exam,Location,Exam Group Key,course,If_Drop,Exam Key,Final Exam flag type by key,Exam key,Exam Group
0,35175,5243547,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,-1,NaN,NaN,NaN,NaN
1,9918,5014298,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,-1,NaN,NaN,NaN,NaN
2,48710,4864791,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,-1,NaN,NaN,NaN,NaN
3,29713,5263561,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,-1,NaN,NaN,NaN,NaN
4,24658,5248498,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,-1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44209,34404,5052204,2839,4496,359151,1,1,102,YORUB,1109,...,YORUB-1109-SEM-102-4496,Y,ITH,NaN,YORUB-1109,-1,NaN,NaN,1-4496,Y&YO
44210,25290,5431809,2839,4496,359151,1,1,102,YORUB,1109,...,YORUB-1109-SEM-102-4496,Y,ITH,NaN,YORUB-1109,-1,NaN,NaN,1-4496,Y&YO
44211,31505,5244975,2839,4496,359151,1,1,102,YORUB,1109,...,YORUB-1109-SEM-102-4496,Y,ITH,NaN,YORUB-1109,-1,NaN,NaN,1-4496,Y&YO
44212,32898,5416547,2839,4496,359151,1,1,102,YORUB,1109,...,YORUB-1109-SEM-102-4496,Y,ITH,NaN,YORUB-1109,-1,NaN,NaN,1-4496,Y&YO


In [7]:
df_all[df_all['If_Drop'] == 1]

,index,ID,Term,Class Nbr,Course ID,Offer Nbr,Session,Section,Subject,Catalog,...,PS Key,Final Exam,Location,Exam Group Key,course,If_Drop,Exam Key,Final Exam flag type by key,Exam key,Exam Group
6,45684,5427835,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,1,NaN,NaN,NaN,NaN
16,27937,5164687,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,1,NaN,NaN,NaN,NaN
41,33063,5057014,2825,17503,352628,2,1,001,AAS,2130,...,AAS-2130-LEC-1-17503,Y,ITH,1-7993C,AAS-2130,1,NaN,NaN,NaN,NaN
84,48649,5311600,2825,20152,361209,1,1,001,AEM,1200,...,AEM-1200-LEC-1-20152,Y,ITH,20152,AEM-1200,1,NaN,NaN,NaN,NaN
111,45412,5220949,2825,20152,361209,1,1,001,AEM,1200,...,AEM-1200-LEC-1-20152,Y,ITH,20152,AEM-1200,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44200,1343,5461805,2839,16850,372881,1,1,001,VTPEH,6195,...,VTPEH-6195-LEC-1-16850,Y,ITH,NaN,VTPEH-6195,1,NaN,NaN,1-16850,Y&YO
44201,1184,4454622,2839,16850,372881,1,1,001,VTPEH,6195,...,VTPEH-6195-LEC-1-16850,Y,ITH,NaN,VTPEH-6195,1,NaN,NaN,1-16850,Y&YO
44202,3184,2763218,2839,16850,372881,1,1,001,VTPEH,6195,...,VTPEH-6195-LEC-1-16850,Y,ITH,NaN,VTPEH-6195,1,NaN,NaN,1-16850,Y&YO
44203,265,4048603,2839,16850,372881,1,1,001,VTPEH,6195,...,VTPEH-6195-LEC-1-16850,Y,ITH,NaN,VTPEH-6195,1,NaN,NaN,1-16850,Y&YO
